In [16]:
# Importing some required libraries
import os
import asyncio
from dotenv import load_dotenv
from google import genai

In [17]:
# this is to load the google api key!
load_dotenv()

True

#### Consider a Scenario where you have a user ID and you want to fetch the user's details from a database. This is not possible with a simple LLM call as it does not have access to your database. However, an agent can be designed to first fetch the user details from the database and then use the LLM to format or process that information.

#### Lets first try out a scenario where we just make a normal LLM call to get the user details related to the given user ID. and see how the model responds!

In [18]:
prompt = "What is the current subscription status for user_id 8851?"

In [19]:
client = genai.Client()
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
)

In [20]:
# Response from gemini-2.5-flash LLM.
print(response.text)

I cannot provide you with the current subscription status for user_id 8851.

As an AI, I don't have access to real-time databases, private user information, or the internal systems of any specific service or company.

To find out the subscription status for user_id 8851, you would need to:

1.  **Log in to the specific service or platform** where that user ID is registered.
2.  **Navigate to the 'Account Settings,' 'Subscription,' or 'Billing' section.**
3.  If you are the account holder, the status should be displayed there. If you're encountering an issue, you may need to **contact the customer support of that service directly.**


#### Now lets go Agentic, lets define a tool that can fetch user details from a mock database and then use the LLM to process that information.

In [ ]:
# Using Google's agent development kit for easy agent creation.
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

In [ ]:
# define a custom tool:

def fetch_user_subscrption_status_from_db(user_id: int) -> str:
    """
    Look up a user's subscription status in the internal database.
    Use this tool when asked about subscription status or account details.
    
    Args:
        user_id: The integer ID of the user.
    """
    # Simulate a database lookup ~ configure or write logic to connect to DB
    mock_db = {
        8851: "Active - Premium Tier (Expires 2025-12-31)",
        9999: "Expired"
    }
    result = mock_db.get(user_id, "User Not Found")
    print(f"[System] Agent is using tool: fetch_user_subscrption_status_from_db({user_id})")
    print(f"[System] Tool returned: {result}\n")
    return result

In [32]:
agent = Agent(
    name="check_subscription_database",
    model=Gemini(
        model="gemini-2.5-flash-lite"
    ),
    description="Look up a user's subscription status in the internal database. Use this when asked about subscription status",
    instruction="You are a helpful assistant who answers questions on users subscription satatus, answer politely",
    tools=[fetch_user_subscrption_status_from_db],
)

In [35]:
runner = InMemoryRunner(agent=agent,app_name="agents")

In [34]:
response = await runner.run_debug(
    prompt
)




 ### Created new session: debug_session_id

User > What is the current subscription status for user_id 8851?



[System] Agent is using tool: fetch_user_subscrption_status_from_db(8851)
[System] Tool returned: Active - Premium Tier (Expires 2025-12-31)

check_subscription_database > The subscription status for user_id 8851 is currently Active - Premium Tier, and it is set to expire on 2025-12-31.
